In [ ]:
import pandas as pd
import numpy as np
import sys, os
import matplotlib.pyplot as plt
import concurrent.futures
import importlib
import seaborn as sns
import scienceplots
import plot_loaders
import plot_utils

plt.style.use(["science","ieee"])

In [ ]:
ORDERINGS = {
    "noise_level": [
        x[0] for x in sorted(plot_loaders.NOISES_MAPPING.items(), key=lambda x: x[1])
    ][::1],
    "variant": ["nonoise", "zerosum", "muffliato"],
    "avgsteps": [i for i in range(20)][::1],
    "topology_type": ["static", "dynamic"],
    "additional_attribute": ["nonoise", "muffliato","selfnoise","noselfnoise"]
}
print(ORDERINGS)

# CIFAR Additional runs with seeds:
# target_dir = "my_results/icml_experiments/additional_cifar10_v2/"
# ORDERINGS = {
#     "noise_level" : ["nonoise","4th","64th"],
#     "variant":["nonoise","zerosum","muffliato"],
#     "avgsteps": ["10", "1"],
#     "topology_type": ["static","dynamic"],
# }


# Muffliato experiments on CIFAR with multiple avg steps
# target_dir = "my_results/icml_experiments/muffliato_avgsteps"
# ORDERINGS = {
#     "noise_level" : ["4th","16th","64th"],
#     "variant":["muffliato","zerosum"],
#     "avgsteps": ["20", "15", "10","5", "1"].reverse(),
#     "topology_type": ["static","dynamic"],
# }


# S&P Femnist experiments
# target_dir = "my_results/femnist_10shards_correctedrandomness/"

# MovieLens Experiments
# target_dir = "my_results/movielens_full/"

# Renewed CIFAR
target_dir= "my_results/cifar_longer/"

# Femnist full experiments
# target_dir = "my_results/femnistLabelSplit_10shards_renewed/"

# Muffliato with less averaging rounds
# target_dir = "my_results/cifar_muffliato_shorteravgrounds/"

linestyles = {"ZeroSum": "--"}

fontsize = 20

alpha = 0.1

save_directory = "assets/"

In [ ]:
# Load the file list
experiments_dict = plot_loaders.get_full_path_dict(target_dir)
experiments_attributes = plot_loaders.get_experiments_dict(experiments_dict)

experiments_attributes_df = pd.DataFrame({})
for experiment_name, experiment_attribute in sorted(experiments_attributes.items()):
    print(experiment_name)
    experiments_attributes_df = pd.concat(
        [
            experiments_attributes_df,
            pd.DataFrame(experiment_attribute, index=[experiment_name]),
        ]
    )
# print(experiments_attributes_df.columns)

# print(experiments_attributes["4447548_muffliato_128nodes_10avgsteps_64th_dynamic_seed105"])
print(experiments_dict)
# for seed in range(91,105):
#     print(f"Seed: {seed:3.0f}:", end= "")
#     current_seed = experiments_attributes_df[experiments_attributes_df["seed"] == f"seed{seed}"]
#     current_variants = list(current_seed["variant"])
#     current_topologies = list(current_seed["topology_type"])
#     print(sorted(list(zip(current_variants,current_topologies))))

In [ ]:
importlib.reload(plot_loaders)
loaded_data = {}


with concurrent.futures.ProcessPoolExecutor(max_workers=20) as executor:
    futures = {}
    for name, filepath in sorted(experiments_dict.items()):
        # plot_loaders.load_data_element(filepath)
        futures[name] = executor.submit(plot_loaders.load_data_element, filepath)
    for name, result in futures.items():
        print(f"Adding data from {name}")
        res = result.result()
        if res.empty:
            print(f"Empty result for {name}")
        loaded_data[name] = res
# print(loaded_data)
print("Finished loading all data")
for column in loaded_data[name]:
    print(column)


## Agregate data

In [ ]:
# Load at once since we'll pop from the dict, this should sanitize the for loop.
for experiment_name, experiment_data in list(loaded_data.items()):
    if experiment_data.empty:
        print(f"Deleting {experiment_name} as it did not load")
        loaded_data.pop(experiment_name)
        experiments_attributes.pop(experiment_name)


importlib.reload(plot_loaders)
# For most of the attacks
general_agg_methods = [
    "mean",
    "std",
    "sum",
    "min",
    "max",
    plot_loaders.percentile(0.5),
    plot_loaders.percentile(0.95),
]
columns_to_agg = [
    "train_loss",
    "test_loss",
    "test_acc",
    "total_bytes",
    "total_meta",
    "total_data_per_n",
]

# Comment from here if some results are not there.
# columns_to_agg += [
#     "roc_auc",
#     "roc_auc_balanced",
#     "50auc-distance",
#     "50auc-distance_balanced",
# ]
columns_to_agg += [
    "classifier_roc_auc",
]
columns_to_agg += [
    f"classifier_tpr_at_fpr{fpr}" for fpr in [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]
]

linkability_aggregators = {
    "linkability_top1": [plot_loaders.count_percentage_success],
    "linkability_top5": [plot_loaders.count_percentage_success],
    "linkability_real_rank": ["median", "min", "max"],
}
# Remember to comment this line if you want linkability results.
linkability_aggregators = {}

to_start_avg = []
# to_start_avg = [
#     "roc_auc mean",
#     "linkability_top1 count_percentage_success",
#     "roc_auc max",
# ]
general_aggregator = {column: general_agg_methods for column in columns_to_agg}

general_aggregator.update(linkability_aggregators)

formatted_data = {}
for name, data in sorted(loaded_data.items()):
    print(f"Formatting {name}")
    current_res = plot_loaders.format_data(
        data,
        name,
        columns_to_agg,
        linkability_aggregators,
        general_aggregator,
        to_start_avg,
    )
    if current_res.empty:
        print(f"WARNING: Removed experiment {name}!!!!")
        experiments_attributes.pop(name)
    else:
        formatted_data[name] = current_res

formatted_data[list(formatted_data)[0]]

In [ ]:
print([x for x in formatted_data[list(formatted_data)[0]].columns if "classifier" in x])
formatted_data[list(formatted_data)[0]]["communication_step"]

In [ ]:
importlib.reload(plot_loaders)
importlib.reload(plot_utils)
static_filter = {"topology_type": ["static"]}
dynamic_filter = {"topology_type": ["dynamic"]}

noisy_filter = {"variant": ["muffliato", "zerosum"]}

static_experiments = plot_loaders.filter_attribute(
    experiments_attributes, static_filter
)
dynamic_experiments = plot_loaders.filter_attribute(
    experiments_attributes, dynamic_filter
)

nonoise_experiments = plot_loaders.filter_attribute(
    experiments_attributes, {"variant": ["nonoise"]}
)

noisy_experiments = plot_loaders.filter_attribute(
    experiments_attributes, {"variant": ["muffliato", "zerosum"]}
)


# To display full experiments
display_attributes = {
    "hue": "noise_level",
    "style":["variant","avgsteps"],
    "col" : "topology_type",
}

# To display experiment parameters tuning.
# display_attributes = {
#     "hue": "lr",
#     "style":"local_rounds",
#     "col" : "model",
# }

# To display muffliato results
# display_attributes = {
#     "hue": "avgsteps",
#     "style": "variant",
#     "col": "noise_level",
# }


for experiment in experiments_attributes:
    print(experiment)
    # print(formatted_data[experiment])



for y_axis, title in [
    ("train_loss mean", "All experiments train losses evolution"),
    ("test_acc mean", "All experiments accuracies evolution"),
]:
    plot = plot_utils.plot_all_experiments(
        data=formatted_data,
        experiments=noisy_experiments,
        display_attributes=display_attributes,
        plot_name=title,
        column_name=y_axis,
        x_axis="iteration",
        save_directory=save_directory,
        orderings = ORDERINGS,
        baseline={nonoise_experiments[0]:formatted_data[nonoise_experiments[0]]},
    )

Doing the same plot, but with communication steps instead of iterations (and thus gradient descent)

In [ ]:
importlib.reload(plot_utils)
for y_axis, title in [
    ("train_loss mean", "All experiments train losses evolution - communication steps"),
    ("test_acc mean", "All experiments accuracies evolution - communication steps"),
]:
    plot = plot_utils.plot_all_experiments(
        data=formatted_data,
        experiments=experiments_attributes,
        display_attributes=display_attributes,
        plot_name=title,
        column_name=y_axis,
        x_axis="communication_step",
        save_directory=save_directory,
        baseline={nonoise_experiments[0]:formatted_data[nonoise_experiments[0]]},
        orderings = ORDERINGS,
    )


# Attack results
## In the static case:

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {"variant":["nonoise","zerosum"]}
# zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["static"]}
# zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["dynamic"]}

display_attributes = {
    "hue":"noise_level",
    "style": "additional_attribute",
    "size": "variant",
    "col": "topology_type",
}

zerosum_experiments = plot_loaders.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)

plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Attack AUC of Zerosum with and without self noise",
    column_name="roc_auc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

# Results vs Muffliato

In [ ]:
importlib.reload(plot_utils)

zerosum_attributes = {
    "variant":["zerosum","nonoise"],
    # "topology_type":["static"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"],
    # "avgsteps": ["1avgsteps","10avgsteps"],
    "additional_attribute": ["nonoise","selfnoise"]}
muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["static"],
    # "topology_type":["dynamic"],
    "avgsteps":["10avgsteps"],
}

current_attributes= [zerosum_attributes,muffliato_attributes]

# display_attributes = {
#     "hue":"noise_level",
#     "style":"variant",
#     # "size":"variant", # For some reason having both style and size breaks in this case.
#     "col":"topology_type",
# }

display_attributes = {
    "hue":["noise_level", "variant"],
    "style":["noise_level", "variant"],
    # "size":"variant", # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)


plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Accuracy of Zerosum and Muffliato",
    column_name="test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()
plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Attack AUC of Zerosum and Muffliato",
    column_name="roc_auc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

### Smaller plots to have an easier time to read the information

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","1th","2th","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["1th","2th","4th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]


# display_attributes = {
#     "hue":"noise_level",
#     "style":"variant",
#     # "size":"variant", # For some reason having both style and size breaks in this case.
#     "col":"topology_type",
# }

display_attributes = {
    "hue":["noise_level", "variant"],
    "style":["noise_level", "variant"],
    # "size":"variant", # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)



plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="High noise:Accuracy of Zerosum and Muffliato with and without self noise",
    column_name="test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()

plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="High noise:Attack AUC of Zerosum and Muffliato with and without self noise",
    column_name="roc_auc max",
    save_directory=save_directory,
    orderings=ORDERINGS,
)
plt.ylim(top=1)

# Plot the middle noises
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","8th","16th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["8th","16th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

plt.figure()
plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Middle noise:Accuracy of Zerosum and Muffliato with and without self noise",
    column_name="test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()
plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Middle noise:Attack AUC of Zerosum and Muffliato with and without self noise",
    column_name="roc_auc max",
    save_directory=save_directory,
    orderings=ORDERINGS,
)
plt.ylim(top=1)

# Plot the low noises
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","32th","64th","128th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["32th","64th","128th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

plt.figure()
plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Low noise:Accuracy of Zerosum and Muffliato with and without self noise",
    column_name="test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()
plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Low noise:Attack AUC of Zerosum and Muffliato with and without self noise",
    column_name="roc_auc max",
    save_directory=save_directory,
    orderings=ORDERINGS
)
plt.ylim(top=1)



# Trying out scatterplots

## Aggregated plots:

In [ ]:
test_data = formatted_data[list(formatted_data)[0]]
print(test_data.columns)
test_data[["classifier_attacked_information","classifier_attacker_model","classifier_attacker_fraction","classifier_attacker_dataset_mode"]]

In [ ]:
importlib.reload(plot_loaders)
importlib.reload(plot_utils)
# Aggregation of each experiments (across iterations)
x_method = "mean"
y_method = "max" # We often consider the maximum accuracy)
# y_method = "mean"
# y_method = "min"
filter_classifier_result = ["mid","SimpleAttacker",0.7,"global"]

zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["static"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["static"],
    "avgsteps": ["5avgsteps"],
}

name_formater = {
    "test_acc mean max":"Maximum test accuracy [%]",
    "nonoise":"test"
}


current_attributes = [zerosum_attributes,muffliato_attributes]

display_attributes = {
    "hue":"variant",
    "style":"variant",
    "size":"variant", # For some reason having both style and size breaks in this case.
    # "col":"topology_type",# Cannot have col argument for the lineplot function
}
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)


# print([x for x in formatted_data[experiment].columns if "test_acc" in x])
# plot_utils.scatter_averaged_experiments(
#     formatted_data,
#     zerosum_experiments,
#     display_attributes,
#     "Accuracy vs AUC evolution accross entire runs-static topology",
#     "test_acc mean",
#     # x_axis_name="50auc-distance mean",
#     x_axis_name="roc_auc mean",
#     save_directory=save_directory,
#     y_method=y_method,
#     x_method=x_method,
#     orderings=ORDERINGS,
#     name_formater=name_formater,
# )

# plt.figure()
# plot_utils.scatter_averaged_experiments(
#     formatted_data,
#     zerosum_experiments,
#     display_attributes,
#     "Accuracy vs Linkability attack for the entire run-static topology",
#     y_axis_name="test_loss mean",
#     x_axis_name="linkability_top1 count_percentage_success",
#     save_directory=save_directory,
#     y_method=y_method,
#     x_method=x_method,
#     orderings=ORDERINGS,
#     name_formater=name_formater,
# )

plt.figure()
plot_utils.scatter_averaged_experiments(
    plot_loaders.filter_classifier(formatted_data,*filter_classifier_result),
    zerosum_experiments,
    display_attributes,
    f"Classifier attack for the entire run\n {filter_classifier_result}",
    y_axis_name="test_acc mean",
    x_axis_name="classifier_roc_auc mean",
    save_directory=save_directory,
    y_method=y_method,
    x_method=x_method,
    orderings=ORDERINGS,
    name_formater=name_formater,
)
plt.figure()
plot_utils.scatter_averaged_experiments(
    plot_loaders.filter_classifier(formatted_data,*filter_classifier_result),
    zerosum_experiments,
    display_attributes,
    f"Classifier attack for the entire run\n {filter_classifier_result}",
    y_axis_name="test_acc mean",
    x_axis_name="classifier_tpr_at_fpr0.1 mean",
    save_directory=save_directory,
    y_method=y_method,
    x_method=x_method,
    orderings=ORDERINGS,
    name_formater=name_formater,
)


In [ ]:
# Look at the dynamic case
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"],
}
current_attributes = [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

plt.figure()
plot_utils.scatter_averaged_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Accuracy vs AUC evolution accross entire runs-dynamic topology",
    "test_acc mean",
    "roc_auc mean",
    save_directory=save_directory,
    y_method=y_method,
    x_method=x_method,
    orderings=ORDERINGS,
    name_formater=name_formater,
)

plt.figure()
plot_utils.scatter_averaged_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Accuracy vs Linkability attack for the entire run-dynamic topology",
    y_axis_name="test_acc mean",
    x_axis_name="linkability_top1 count_percentage_success",
    save_directory=save_directory,
    y_method=y_method,
    x_method=x_method,
    orderings=ORDERINGS,
    name_formater=name_formater,
)

plt.figure()
plot_utils.scatter_averaged_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Accuracy vs Linkability top5 attack for the entire run-dynamic topology",
    y_axis_name="test_acc mean",
    x_axis_name="linkability_top5 count_percentage_success",
    save_directory=save_directory,
    y_method=y_method,
    x_method=x_method,
    orderings=ORDERINGS,
    name_formater=name_formater,
)

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise"],
    "topology_type":["static"],
    "avgsteps": ["1avgsteps"],
    # "noise_level":[None,"2th","4th"]
}


zerosum_experiments = plot_loaders.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)


experiment = zerosum_experiments[0]
experiment_data = formatted_data[experiment] 

sns.set_theme()
sns.relplot(data=experiment_data, x="roc_auc mean", y="test_acc mean", hue="iteration")

title = "Evolution of the unnoised test accuracy and AUC-static topology"
plt.title(title)
plt.savefig(save_directory + title.replace(" ","_") +".pdf")

## Better experiment display (selected noises levels)

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["64th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]

display_attributes = {
    "hue":["variant","noise_level","avgsteps"],
    "style":["variant","noise_level","avgsteps"],
    "size":["variant","noise_level","avgsteps"], # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

order_mapper = {"muffliato":2,"nonoise":0,"zerosum":1}
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
zerosum_experiments = sorted(zerosum_experiments, key = lambda x : order_mapper[x.split("_")[1]])
for experiment in zerosum_experiments:
    print(experiment)
# zerosum_experiments = [zerosum_experiments[i] for i in [0,1,4,5,2,3] ]



plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Selected noise:Accuracy of Zerosum and Muffliato",
    column_name="test_acc mean",
    save_directory=save_directory,
    orderings = ORDERINGS,
)


plt.figure()

plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Selected noise:Attack AUC of Zerosum and Muffliato",
    column_name="roc_auc mean_start_avg",
    save_directory=save_directory,
    orderings = ORDERINGS,
)

plt.figure()

plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Selected noise:Linkability attack ac of Zerosum and Muffliato",
    column_name="linkability_top1 count_percentage_success_start_avg",
    save_directory=save_directory,
    orderings = ORDERINGS,
)

plt.figure()

#Needs to only have hue since we can't have other attributes in jointplots
display_attributes = {
    "hue":["variant","noise_level","avgsteps"],
}

zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["static"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["static"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["64th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)
print("-"*40)


plot_utils.scatter_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise:Accuracy vs AUC-static",
    "roc_auc mean",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)
plt.figure()
plot_utils.scatter_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise:Accuracy vs Linkability sucess rate-static",
    "linkability_top1 count_percentage_success",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","1th","2th","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["1th","2th","4th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]


display_attributes = {
    "hue":"noise_level",
    "style":"variant",
    "size":"avgsteps", # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)



plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="High noise:Accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    column_name="test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()

plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="High noise:Linkability attack accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    column_name="linkability_top1 count_percentage_success",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

# Plot the middle noises
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","8th","16th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["8th","16th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

plt.figure()
plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Middle noise:Accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    column_name="test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()
plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Middle noise:Linkability attack accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    column_name="linkability_top1 count_percentage_success",
    save_directory=save_directory,
    orderings=ORDERINGS,
)


# Plot the low noises
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","32th","64th","128th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["32th","64th","128th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

plt.figure()
plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Low noise:Accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    column_name="test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

plt.figure()
plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Low noise:Linkability attack accuracy of Zerosum and Muffliato with and without self noise-dynamic topology",
    column_name="linkability_top1 count_percentage_success",
    save_directory=save_directory,
    orderings=ORDERINGS,
)
plt.ylim(top=1)



# Start average trial displays

In [ ]:
importlib.reload(plot_utils)
zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    # "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["64th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]

display_attributes = {
    "hue":["variant","noise_level","avgsteps"],
    "style":["variant","noise_level","avgsteps"],
    "size":["variant","noise_level","avgsteps"], # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

order_mapper = {"muffliato":2,"nonoise":0,"zerosum":1}
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
zerosum_experiments = sorted(zerosum_experiments,key = lambda x : order_mapper[x.split("_")[1]])
for experiment in sorted(zerosum_experiments):
    print(experiment)



plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Selected noise: Accuracy of Zerosum and Muffliato",
    column_name="test_acc mean",
    save_directory=save_directory,
    orderings = ORDERINGS,
)


plt.figure()

plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Selected noise: Threshold AUC rolling average of Zerosum and Muffliato",
    column_name="roc_auc mean_start_avg",
    save_directory=save_directory,
    orderings = ORDERINGS,
)

plt.figure()

plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Selected noise: Linkability attack rolling average ac of Zerosum and Muffliato",
    column_name="linkability_top1 count_percentage_success_start_avg",
    save_directory=save_directory,
    orderings = ORDERINGS,
)

plt.figure()

#Needs to only have hue since we can't have other attributes in jointplots
display_attributes = {
    "hue":["variant","noise_level","avgsteps"],
}

zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["static"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["static"],
    "avgsteps": ["10avgsteps"], 
    "noise_level":["64th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)
print("-"*40)


plot_utils.scatter_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise: Accuracy vs Threshold AUC rolling average-static",
    "roc_auc mean_start_avg",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)
plt.figure()
plot_utils.scatter_all_experiments(
    formatted_data,
    zerosum_experiments,
    display_attributes,
    "Selected noise: Accuracy vs Linkability sucess rate rolling average-static",
    "linkability_top1 count_percentage_success_start_avg",
    "test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

In [ ]:
last_expe = list(formatted_data.keys())[-1]
print(formatted_data[last_expe]["noise_level"])

# Display the test loss and test accuracy of inflexion point.

In [ ]:
importlib.reload(plot_utils)

muffliato_attributes = {
    "variant":["zerosum"],
    "additional_attribute":["selfnoise"],
    # "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "noise_level":["6th",
        "7th",
        "8th",
        "16th",
        "32th","64th","128th"]
}
current_attributes= [muffliato_attributes]

display_attributes = {
    "hue":["variant","noise_level_value"],
    # "style":["variant","noise_level","avgsteps"],
    # "size":["variant","noise_level","avgsteps"], # For some reason having both style and size breaks in this case.
    "col":"topology_type",
}

muffliato_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in muffliato_experiments:
    print(experiment)

plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=muffliato_experiments,
    display_attributes=display_attributes,
    plot_name="Inflexion point: test_acc of Muffliato",
    column_name="test_acc mean",
    save_directory=save_directory,
    # orderings = ORDERINGS,
)
plt.figure()
plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=muffliato_experiments,
    display_attributes=display_attributes,
    plot_name="Inflexion point: test_loss of Muffliato",
    column_name="test_loss mean",
    save_directory=save_directory,
    # orderings = ORDERINGS,
)

# Data displays

In [ ]:
importlib.reload(plot_utils)

target_accuracy = 50


zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["static"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    # "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["static"],
    "avgsteps": ["10avgsteps"], 
    # "noise_level":["64th"]
}
current_attributes= [zerosum_attributes,muffliato_attributes]

zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)
order = ORDERINGS["noise_level"]
order_mapper = {"muffliato":2,"nonoise":1,"zerosum":0}
zerosum_experiments = sorted(zerosum_experiments, key = lambda x : order_mapper[x.split("_")[1]])

plot_utils.plot_communication(formatted_data,zerosum_experiments,target_accuracy,f"Communications to reach {target_accuracy}% accuracy-static",save_directory=save_directory,order=order)

zerosum_attributes = {
    "variant":["nonoise","zerosum"],
    "topology_type":["dynamic"],
    "avgsteps": ["1avgsteps"], 
    "additional_attribute": ["nonoise","selfnoise"],
    # "noise_level":["nonoise","4th"]
}

muffliato_attributes = {
    "variant":["muffliato"],
    "topology_type":["dynamic"],
    "avgsteps": ["10avgsteps"], 
    # "noise_level":["64th"]
}

current_attributes= [zerosum_attributes,muffliato_attributes]
zerosum_experiments = plot_loaders.filter_attribute_list(experiments_attributes,current_attributes)

order = ORDERINGS["noise_level"]
order_mapper = {"muffliato":2,"nonoise":0,"zerosum":1}
zerosum_experiments = sorted(zerosum_experiments, key = lambda x : order_mapper[x.split("_")[1]])

plt.figure()
plot_utils.plot_communication(formatted_data,zerosum_experiments,target_accuracy,f"Communications to reach {target_accuracy}% accuracy-dynamic",save_directory=save_directory,order=order)


# Legacy content
Old cells that are mostly in the way, and that I won't execute anymore

## Noise cancellation vs no noise cancellation

In [ ]:

importlib.reload(plot_utils)
zerosum_attributes = {"variant":["nonoise","zerosum"]}
# zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["static"]}
# zerosum_attributes = {"variant":["nonoise","zerosum"],"topology_type":["dynamic"]}

display_attributes = {
    "hue":"noise_level",
    "style": "additional_attribute",
    "size": "variant",
    "col":"topology_type"
}

zerosum_experiments = plot_loaders.filter_attribute(experiments_attributes,zerosum_attributes)
for experiment in sorted(zerosum_experiments):
    print(experiment)

plot_utils.plot_all_experiments(
    data=formatted_data,
    experiments=zerosum_experiments,
    display_attributes=display_attributes,
    plot_name="Test accuracy evolution of ZeroSum with and without self noise",
    column_name="test_acc mean",
    save_directory=save_directory,
    orderings=ORDERINGS,
)

